<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/VALL_E_X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VALL_E_X による音声合成

参考：https://medium.com/axinc/vall-e-x-%E5%86%8D%E5%AD%A6%E7%BF%92%E4%B8%8D%E8%A6%81%E3%81%A7%E5%A3%B0%E8%B3%AA%E3%82%92%E5%A4%89%E6%9B%B4%E3%81%A7%E3%81%8D%E3%82%8B%E9%9F%B3%E5%A3%B0%E5%90%88%E6%88%90%E3%83%A2%E3%83%87%E3%83%AB-977efc19ac84

## 従来の音声合成
これはこれでちゃんとしている。



In [ ]:
!pip install gTTs pydub yt-dlp

### テキスト ->  音声

In [ ]:
from gtts import gTTS
from IPython.display import Audio

message = 'みなさん、こんにちは。私は音声合成で喋っています。'
tts = gTTS(message, lang='ja')
tts.save('gTTS_test.mp3')
Audio('gTTS_test.mp3', autoplay=True)

## VALL-E-X のインストール

In [ ]:
!git clone https://github.com/Plachtaa/VALL-E-X.git
%cd VALL-E-X
!pip install -r requirements.txt

### インポート

In [ ]:
from utils.generation import SAMPLE_RATE, generate_audio, preload_models
from scipy.io.wavfile import write as write_wav

### モデルのロード

In [ ]:
preload_models()

### 音声合成（英語）

In [ ]:
text_prompt = """
    This is a pen.
"""
audio_array = generate_audio(text_prompt)

write_wav("/content/english.wav", SAMPLE_RATE, audio_array)
Audio(audio_array, rate=SAMPLE_RATE, autoplay=True)

### 音声合成（日本語）

In [ ]:
text_prompt = """
    ひきにくでーーす．
"""
audio_array = generate_audio(text_prompt)

write_wav("/content/japanese.wav", SAMPLE_RATE, audio_array)
Audio(audio_array, rate=SAMPLE_RATE, autoplay=True)

### 音声の選択
https://github.com/Plachtaa/VALL-E-X/tree/master/presets

In [ ]:
text_prompt = """
    これは cafe モデルでの音声です。
"""

audio_array = generate_audio(text_prompt, prompt="cafe")

write_wav("/content/sample.wav", SAMPLE_RATE, audio_array)
Audio(audio_array, rate=SAMPLE_RATE, autoplay=True)

# 本番：Voice Cloning（音声複製）
自分で用意した音声データを使って、同じ声で喋らせます。

### 準備：音声の調達
youtube から持ってきます。

In [ ]:
video_url = 'https://www.youtube.com/watch?v=50kENDkk01U' #@param {type:"string"}
start_time = 14 #@param {type:"integer"}
end_time = 20 #@param {type:"integer"}

In [ ]:
from yt_dlp import YoutubeDL

download_resolution = 360
input_clip_path = '/content/input_clip.mp4'
input_audio_path = '/content/input_clip.wav'

def set_download_ranges(info_dict, self):
    duration_opt = [{
        'start_time': start_time,
        'end_time': end_time
    }]
    return duration_opt

# 動画ダウンロード --> input_clip.mp4
ydl_opts = {'format': f'best[height<={download_resolution}]', 'overwrites': True, 'outtmpl': input_clip_path, 'download_ranges': set_download_ranges}
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

# 音声だけ取り出し --> input_clip.mp3
!yes|ffmpeg -i /content/input_clip.mp4 /content/input_clip.wav
Audio('/content/input_clip.wav', autoplay=True)

In [ ]:
from utils.prompt_making import make_prompt

# 先ほど生成した音声を使う
make_prompt(name="sample", audio_prompt_path="/content/input_clip.wav",
                transcript="勝手に祭り上げ、勝手に諦め、そして勝手にコンプレックスを抱く。")

text_prompt = """
    こんにちは。阿部寛です。
"""
audio_array = generate_audio(text_prompt, prompt="sample")

write_wav("/content/sample2.wav", SAMPLE_RATE, audio_array)
Audio(audio_array, rate=SAMPLE_RATE, autoplay=True)

## 右クリックでダウンロードできない人用
Google Drive に sample2.wav というファイルができるので、それをダウンロードして提出。

In [ ]:
# Googleドライブのマウント
from google.colab import drive
drive.mount("/content/drive")
!cp /content/sample2.wav /content/drive/MyDrive/sample2.wav